<a href="https://colab.research.google.com/github/vishnuvarakala/Churn_prediction/blob/main/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tqdm import tqdm




from keras.models import Sequential
from keras.layers import LSTM, GRU,SimpleRNN
from tensorflow.keras.layers import Activation, Dense, Dropout
from keras.layers import Embedding
from keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:



file_path = "/content/drive/My Drive/Colab Notebooks/jigsaw-toxic-comment-train.csv"
file_path1 = "/content/drive/My Drive/Colab Notebooks/test.csv"
file_path2 = "/content/drive/My Drive/Colab Notebooks/validation.csv"

#/content/glove.840B.300d.zip

train = pd.read_csv(file_path)
test = pd.read_csv(file_path1)
validation = pd.read_csv(file_path2)




In [4]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)


In [5]:
train = train.loc[:12000,:]
train.shape

(12001, 3)

In [6]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values,
                                                  stratify=train.toxic.values,
                                                  random_state=42,
                                                  test_size=0.2, shuffle=True)

In [7]:
from tensorflow.keras.preprocessing import sequence

token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# Zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index


In [9]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                 300,
                 input_length=max_len))
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1500, 300)         13049100  
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 100)               40100     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 13089301 (49.93 MB)
Trainable params: 13089301 (49.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64)



Epoch 1/5
150/150 [==============================] - 218s 1s/step - loss: 0.3008 - accuracy: 0.9024
Epoch 2/5
150/150 [==============================] - 207s 1s/step - loss: 0.1073 - accuracy: 0.9620
Epoch 3/5
150/150 [==============================] - 202s 1s/step - loss: 0.0135 - accuracy: 0.9966
Epoch 4/5
150/150 [==============================] - 198s 1s/step - loss: 0.0019 - accuracy: 0.9998
Epoch 5/5
150/150 [==============================] - 196s 1s/step - loss: 6.2459e-04 - accuracy: 1.0000


In [14]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''

    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

from sklearn.metrics import roc_auc_score

scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

76/76 [==============================] - 9s 119ms/step
Auc: 0.84%


In [16]:
#scores_model.append({'Model': 'LSTM','AUC_Score': roc_auc(scores,yvalid)})


In [2]:
!pip install gdown


In [3]:
import gdown

# Define the GloVe embeddings URL and output path
glove_url = 'http://www-nlp.stanford.edu/data/glove.840B.300d.zip'
output_path = 'glove.840B.300d.zip'

# Download the GloVe embeddings file
gdown.download(glove_url, output_path, quiet=False)


Downloading...
From: http://www-nlp.stanford.edu/data/glove.840B.300d.zip
To: /content/glove.840B.300d.zip
100%|██████████| 2.18G/2.18G [06:50<00:00, 5.31MB/s]


'glove.840B.300d.zip'

In [4]:
import zipfile

#/content/drive/My Drive/Colab Notebooks/
# Define the path to the directory where you want to extract the files
extract_path = '/content/drive/My Drive/Colab Notebooks/'

# Unzip the GloVe embeddings file
with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [5]:
import os

# List the files in the extraction directory
files = os.listdir(extract_path)
print("Files in the extraction directory:")
for file in files:
    print(file)


Files in the extraction directory:
glove.840B.300d.txt


In [ ]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('/content/drive/My Drive/Colab Notebooks/glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

1260822it [01:58, 7644.81it/s]

In [12]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

  0%|          | 0/43496 [00:00<?, ?it/s]


NameError: ignored

In [17]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                      300,
                      weights=[embedding_matrix],
                      input_length=max_len,
                      trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

model.summary()

NameError: ignored